# MNIST

This notebook explores transfer learning techniques on MNIST.

## Change working directory to project root

In [ ]:
import os
ROOT_DIRECTORIES = {'dogwood', 'tests'}
if set(os.listdir('.')).intersection(ROOT_DIRECTORIES) != ROOT_DIRECTORIES:
    os.chdir('../..')

## Exploration

In [ ]:
from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt
import h5py
import tensorflow as tf
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten

In [ ]:
MNIST_IMAGE_SHAPE = (28, 28)
MAX_PIXEL_VALUE = 255
MODEL_SAVE_DIR = '/tmp/dogwood/mnist'

In [ ]:
(X_train, y_train), (X_test, y_test) = mnist.load_data()
X_train = tf.cast(X_train, tf.float32) / MAX_PIXEL_VALUE
X_test = tf.cast(X_test, tf.float32) / MAX_PIXEL_VALUE
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

In [ ]:
model = Sequential([
    Flatten(input_shape=(MNIST_IMAGE_SHAPE)),
    Dense(128, activation='relu', name='dense_1'),
    Dense(10, activation='softmax', name='dense_2')
])
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['sparse_categorical_accuracy'])

In [ ]:
model.fit(X_train, y_train, epochs=10, batch_size=32)

In [ ]:
model.evaluate(X_test, y_test)

## New models using (sometimes partially) pretrained weights

In [ ]:
Path(MODEL_SAVE_DIR).mkdir(parents=True, exist_ok=True)
model_path = os.path.join(MODEL_SAVE_DIR, 'model.h5')
model.save_weights(model_path)

In [ ]:
with h5py.File(model_path, 'r') as infile:
    print(infile.keys())

In [ ]:
with h5py.File(model_path, 'r') as infile:
    print(infile['dense_1'].keys())

In [ ]:
with h5py.File(model_path, 'r') as infile:
    print(infile['dense_1']['dense_1'].keys())

In [ ]:
with h5py.File(model_path, 'r') as infile:
    print(infile['dense_1']['dense_1']['bias:0'])

In [ ]:
with h5py.File(model_path, 'r') as infile:
    print(infile['dense_1']['dense_1']['kernel:0'])

We can visualize these weights in a 2D plot, although they make more sense in 28 x 28.

In [ ]:
with h5py.File(model_path, 'r') as infile:
    biases_1 = infile['dense_1']['dense_1']['bias:0'][:]
    weights_1 = infile['dense_1']['dense_1']['kernel:0'][:]

In [ ]:
fig, ax = plt.subplots(figsize=(8, 8))
heatmap = ax.pcolor(weights_1.T, cmap=plt.cm.Blues)
cbar = ax.figure.colorbar(heatmap, ax=ax)
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(8, 8))
heatmap = ax.pcolor(np.expand_dims(biases_1, axis=-1), cmap=plt.cm.Blues)
cbar = ax.figure.colorbar(heatmap, ax=ax)
plt.show()

In [ ]:
figs, axes = plt.subplots(ncols=2, figsize=(12, 8))
heatmap_weights = axes[0].pcolor(weights_1.T, cmap=plt.cm.Blues)
heatmap_biases = axes[1].pcolor(np.expand_dims(biases_1, axis=-1), cmap=plt.cm.Blues)
plt.show()